This package is an implementation of "Symbolic Reasoning about Quantum Circuits in Coq." In that paper they implement a way of simpylinf circuits using symbols instead of Matrix multiplication  . Usually when you apply an X gate to a state, you do by represensting the operation as a matrix. In dirac notation, you can simpifly it in a symbolic way. Taking the same example they have in that paper, they implement that sybmplically.  This is way of deriving quantum circuits can be very useful for research who tend to use dirac notation instead of matrix multiplication.

In [1]:
from IPython.display import display, Math, HTML
from sympy.physics.quantum import TensorProduct, qapply
from symboliq.dirac_notation import DiracNotation, cx, h, i, ket_0, x, ket_1
from sympy.physics.quantum.gate import XGate
from sympy.physics.quantum.qubit import Qubit
import sympy
import symboliq
from sympy.core.numbers import Integer

In sympy, you can have a program like 

In [2]:
state = Qubit("0")

res = XGate(0) * state
print(res)

application = qapply(res)
print(application)

X(0)*|0>
|1>


Internally, Sympy applies this qubit to a state via matrix multiplication. It's similar to calling

In [3]:
XGate(0)._apply_operator_Qubit(state)

|1>

Which is implemented to perform matrx multiplication. Below is a simplified version of the implementation

In [4]:
def apply_operator_Qubit(operator, qubits):
    targets = operator.targets
    target_matrix = operator.get_target_matrix()  # returns Matrix([[0, 1], [1, 0]]) for an X gate

    # Find which column of the target matrix this applies to.
    column_index = 0
    n = 1
    for target in targets:
        column_index += n * qubits[target]
        n = n << 1
        column = target_matrix[:, int(column_index)]

    # Now apply each column element to the qubit.
    result = 0
    for index in range(column.rows):
        new_qubit = qubits.__class__(*qubits.args)
        # Flip the bits that need to be flipped.
        for bit, target in enumerate(targets):
            if new_qubit[target] != (index >> bit) & 1:
                new_qubit = new_qubit.flip(target)
        # The value in that row and column times the flipped-bit qubit
        # is the result for that part.
        result += column[index] * new_qubit
    return result

In [5]:
apply_operator_Qubit(XGate(0), state)

|1>

SymboliQ on the other hand uses Dirac Notation

In [6]:
symboliq.qapply(XGate(0) * state)

|1>

In [9]:
steps = symboliq.get_simp_steps(XGate(0) * state)

In [10]:
print(steps)

(0) X(0)*|0>
(1) |0><1|*|0> + |1><0|*|0>
(2) |1>

